In [1]:
import numpy as np
from multiple_factor import DGP2, Inferece2
from dgp import DGP
from inference import Inference

def reject_prob(Xdim, num_factor, sample_size, ntrials=1000):
    phi_tau = np.zeros((ntrials,2))
    for i in range(ntrials):
        dgp = DGP2(num_factor, sample_size, Xdim)
        Y, D, tuple_idx = dgp.Y, dgp.D, dgp.tuple_dix
        inf = Inferece2(Y, D, tuple_idx)
        phi_tau[i,0] = inf.phi_tau
        phi_tau[i,1] = inf.phi_tau_p
    return np.mean(phi_tau, axis=0)

"""
def reject_prob1(n, modelY='1', modelDA='8', ate=0, ntrials=1000):
    phi_tau11s, phi_tau10s, phi_theta1s, phi_theta2s, phi_theta12s = np.zeros(ntrials), np.zeros(ntrials), np.zeros(ntrials), np.zeros(ntrials), np.zeros(ntrials)
    for i in range(ntrials):
        dgp = DGP(modelY,modelDA,n,tau=ate)
        inf = Inference(dgp.Y, dgp.D, dgp.A, modelDA, tuple_idx=dgp.tuple_idx, tau=dgp.tau)
        phi_tau11, phi_tau10, phi_theta1, phi_theta2, phi_theta12 = inf.inference()
        phi_tau11s[i] = phi_tau11
        phi_tau10s[i] = phi_tau10
        phi_theta1s[i] = phi_theta1
        phi_theta2s[i] = phi_theta2
        phi_theta12s[i] = phi_theta12
        break
    return phi_tau11s, phi_tau10s, phi_theta1s, phi_theta2s, phi_theta12s
"""


"""
res, res_p = np.zeros((4,5)), np.zeros((4,5))
for i, q in enumerate([1, 2, 4, 8]):
    for j, k in enumerate([2, 3, 4, 5, 6]):
        p1, p2 = reject_prob(q, k, 1280, 500)
        res[i,j] = p1
        res_p[i,j] = p2
        print("q={}, k={}, p1={},p2={}".format(q,k,p1,p2))
"""

'\nres, res_p = np.zeros((4,5)), np.zeros((4,5))\nfor i, q in enumerate([1, 2, 4, 8]):\n    for j, k in enumerate([2, 3, 4, 5, 6]):\n        p1, p2 = reject_prob(q, k, 1280, 500)\n        res[i,j] = p1\n        res_p[i,j] = p2\n        print("q={}, k={}, p1={},p2={}".format(q,k,p1,p2))\n'

In [2]:
from joblib import Parallel, delayed
import multiprocessing
# what are your inputs, and what operation do you want to
# perform on each input. For example...
qk_pairs = [(q,k) for q in [1,2,4,8,10] for k in [2,3,4,5,6]]
def processInput(qk):
    q, k = qk
    p1, p2 = reject_prob(q, k, 1280, 1000)
    return (q,k,p1,p2)
num_cores = multiprocessing.cpu_count()
results = Parallel(n_jobs=num_cores)(delayed(processInput)(i) for i in qk_pairs)
print(results)

[(1, 2, 0.059, 0.059), (1, 3, 0.051, 0.049), (1, 4, 0.048, 0.046), (1, 5, 0.051, 0.052), (1, 6, 0.038, 0.038), (2, 2, 0.029, 0.025), (2, 3, 0.019, 0.014), (2, 4, 0.008, 0.007), (2, 5, 0.005, 0.005), (2, 6, 0.002, 0.002), (4, 2, 0.009, 0.006), (4, 3, 0.008, 0.006), (4, 4, 0.006, 0.006), (4, 5, 0.008, 0.008), (4, 6, 0.005, 0.005), (8, 2, 0.018, 0.011), (8, 3, 0.011, 0.01), (8, 4, 0.011, 0.011), (8, 5, 0.023, 0.022), (8, 6, 0.025, 0.025), (10, 2, 0.019, 0.013), (10, 3, 0.014, 0.012), (10, 4, 0.012, 0.01), (10, 5, 0.014, 0.012), (10, 6, 0.021, 0.021)]


# [(1, 2, 0.059, 0.059), (1, 3, 0.051, 0.049), (1, 4, 0.048, 0.046), (1, 5, 0.051, 0.052), (1, 6, 0.038, 0.038), (2, 2, 0.029, 0.025), (2, 3, 0.019, 0.014), (2, 4, 0.008, 0.007), (2, 5, 0.005, 0.005), (2, 6, 0.002, 0.002), (4, 2, 0.009, 0.006), (4, 3, 0.008, 0.006), (4, 4, 0.006, 0.006), (4, 5, 0.008, 0.008), (4, 6, 0.005, 0.005), (8, 2, 0.018, 0.011), (8, 3, 0.011, 0.01), (8, 4, 0.011, 0.011), (8, 5, 0.023, 0.022), (8, 6, 0.025, 0.025), (10, 2, 0.019, 0.013), (10, 3, 0.014, 0.012), (10, 4, 0.012, 0.01), (10, 5, 0.014, 0.012), (10, 6, 0.021, 0.021)]

In [6]:
def ate(Xdim, num_factor, sample_size, ntrials=1000):
    ates = np.zeros(ntrials)
    for i in range(ntrials):
        dgp = DGP2(num_factor, sample_size, Xdim)
        Y, D, tuple_idx = dgp.Y, dgp.D, dgp.tuple_dix
        inf = Inferece2(Y, D, tuple_idx)
        ate = inf.tau
        ates[i] = ate
    return ates

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()


def plot_hist(tau1, tau2, tau3, tau4):
    
    fig, (ax_1, ax_2, ax_3, ax_4) = plt.subplots(1, 4, figsize=(20, 5))
    ax_1.hist(tau1, color="b")
    ax_1.set_xlabel("Dim(X)=1, K=2")
    ax_1.set_ylabel("Count")
    ax_1.axvline(x=np.mean(tau1), ymin=0, ymax=3000, color="k", linestyle="--")

    ax_2.hist(tau2, color="b")
    ax_2.set_xlabel("Dim(X)=10, K=2")
    ax_2.set_ylabel("Count")
    ax_2.axvline(x=np.mean(tau2), ymin=0, ymax=3000, color="k", linestyle="--")
    
    ax_3.hist(tau3, color="b")
    ax_3.set_xlabel("Dim(X)=1, K=6")
    ax_3.set_ylabel("Count")
    ax_3.axvline(x=np.mean(tau3), ymin=0, ymax=3000, color="k", linestyle="--")
    
    ax_4.hist(tau4, color="b")
    ax_4.set_xlabel("Dim(X)=10, K=6")
    ax_4.set_ylabel("Count")
    ax_4.axvline(x=np.mean(tau4), ymin=0, ymax=3000, color="k", linestyle="--")
    return fig

In [7]:
tau1 = ate(1, 2, 1280, 1000)
tau2 = ate(10, 2, 1280, 1000)
tau3 = ate(1, 6, 1280, 1000)
tau4 = ate(10, 6, 1280, 1000)

In [ ]:
plot_hist(tau1, tau2, tau3, tau4)